<a href="https://colab.research.google.com/github/HemaSagarKoppusetti/DualityAI-HWH/blob/main/object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Download the dataset
!wget -O hackathon2_train_1.zip "https://storage.googleapis.com/duality-public-share/Datasets/hackathon2_train_1.zip"

# Step 2: Unzip the dataset
!unzip -q hackathon2_train_1.zip -d hackathon2_dataset

# Step 3: Check extracted files
import os

base_path = "hackathon2_dataset"
for root, dirs, files in os.walk(base_path):
    print(f"📂 {root} - {len(files)} files")
    # Show first 5 files in each folder
    print(files[:5])

--2025-09-22 07:42:25--  https://storage.googleapis.com/duality-public-share/Datasets/hackathon2_train_1.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.128.207, 74.125.143.207, 173.194.69.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.128.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6362208415 (5.9G) [application/zip]
Saving to: ‘hackathon2_train_1.zip’

hackathon2_train_1. 100%[===================>]   5.92G  32.5MB/s    in 2m 42s  

2025-09-22 07:45:07 (37.4 MB/s) - ‘hackathon2_train_1.zip’ saved [6362208415/6362208415]

📂 hackathon2_dataset - 0 files
[]
📂 hackathon2_dataset/train_1 - 0 files
[]
📂 hackathon2_dataset/train_1/val1 - 0 files
[]
📂 hackathon2_dataset/train_1/val1/labels - 336 files
['000000131_vlight_uncluttered.txt', '000000052_vlight_uncluttered.txt', '000000272_vcluttered_room.txt', '000000178_vlight_cluttered_room.txt', '000000115_vcluttered_room.txt']
📂 hackathon2_dataset/train_1/

In [2]:
!unzip -q "/content/Hackathon2_scripts.zip" -d "/content/Hackathon2_scripts_extracted"

In [3]:
# In Google Colab:

# Step 1: Download the zip file
!wget -O hackathon2_test1.zip "https://storage.googleapis.com/duality-public-share/Datasets/Hackathon2_test1.zip"

# Step 2: Unzip the downloaded file into a folder
!unzip -q hackathon2_test1.zip -d hackathon2_test1_dataset

# Step 3: Verify contents
import os

base_path = "hackathon2_test1_dataset"
count = 0
for root, dirs, files in os.walk(base_path):
    print(f"Folder: {root} → {len(files)} files")
    if len(files) > 0:
        print("Sample files:", files[:5])
    count += 1
    # optionally stop after first few for speed
    if count >= 3:
        break

--2025-09-22 07:46:18--  https://storage.googleapis.com/duality-public-share/Datasets/Hackathon2_test1.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.218.207, 142.251.31.207, 142.251.18.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.218.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4206767161 (3.9G) [application/zip]
Saving to: ‘hackathon2_test1.zip’

hackathon2_test1.zi 100%[===================>]   3.92G  39.6MB/s    in 1m 49s  

2025-09-22 07:48:07 (37.0 MB/s) - ‘hackathon2_test1.zip’ saved [4206767161/4206767161]

Folder: hackathon2_test1_dataset → 0 files
Folder: hackathon2_test1_dataset/test1 → 0 files
Folder: hackathon2_test1_dataset/test1/labels → 1408 files
Sample files: ['000000265_dark_clutter.txt', '000000125_vlight_unclutter.txt', '000000137_dark_clutter.txt', '000000896_dark_clutter.txt', '000000555_light_unclutter.txt']


In [3]:
!pip install ultralytics
from ultralytics import YOLO
from pathlib import Path
import cv2
import os
import yaml



# Function to predict and save images
def predict_and_save(model, image_path, output_path, output_path_txt):
    # Perform prediction
    results = model.predict(image_path,conf=0.5)

    result = results[0]
    # Draw boxes on the image
    img = result.plot()  # Plots the predictions directly on the image

    # Save the result
    cv2.imwrite(str(output_path), img)
    # Save the bounding box data
    with open(output_path_txt, 'w') as f:
        for box in result.boxes:
            # Extract the class id and bounding box coordinates
            cls_id = int(box.cls)
            x_center, y_center, width, height = box.xywhn[0].tolist()

            # Write bbox information in the format [class_id, x_center, y_center, width, height]
            f.write(f"{cls_id} {x_center} {y_center} {width} {height}\n")


if __name__ == '__main__':

    this_dir = Path('.')
    os.chdir(this_dir)
    with open(this_dir / '/content/Hackathon2_scripts_extracted/Hackathon2_scripts/yolo_params.yaml', 'r') as file:
        data = yaml.safe_load(file)
        if 'test' in data and data['test'] is not None:
            images_dir = Path(data['test']) / 'images'
        else:
            print("No test field found in yolo_params.yaml, please add the test field with the path to the test images")
            exit()

    # check that the images directory exists
    if not images_dir.exists():
        print(f"Images directory {images_dir} does not exist")
        exit()

    if not images_dir.is_dir():
        print(f"Images directory {images_dir} is not a directory")
        exit()

    if not any(images_dir.iterdir()):
        print(f"Images directory {images_dir} is empty")
        exit()

    # Load the YOLO model
    detect_path = this_dir / "runs" / "detect"
    # Added check for directory existence
    if not detect_path.exists():
        print(f"Error: The directory '{detect_path}' was not found.")
        print("Please ensure that the model training cell (cell 77689678) has been executed successfully before running this cell.")
        exit()

    train_folders = [f for f in os.listdir(detect_path) if os.path.isdir(detect_path / f) and f.startswith("train")]
    if len(train_folders) == 0:
        raise ValueError("No training folders found")
    idx = 0
    if len(train_folders) > 1:
        choice = -1
        choices = list(range(len(train_folders)))
        while choice not in choices:
            print("Select the training folder:")
            for i, folder in enumerate(train_folders):
                print(f"{i}: {folder}")
            choice = input()
            if not choice.isdigit():
                choice = -1
            else:
                choice = int(choice)
        idx = choice

    model_path = detect_path / train_folders[idx] / "weights" / "best.pt"
    model = YOLO(model_path)

    # Directory with images
    output_dir = this_dir / "predictions" # Replace with the directory where you want to save predictions
    output_dir.mkdir(parents=True, exist_ok=True)

    # Create images and labels subdirectories
    images_output_dir = output_dir / 'images'
    labels_output_dir = output_dir / 'labels'
    images_output_dir.mkdir(parents=True, exist_ok=True)
    labels_output_dir.mkdir(parents=True, exist_ok=True)

    # Iterate through the images in the directory
    for img_path in images_dir.glob('*'):
        if img_path.suffix not in ['.png', '.jpg']:
            continue
        output_path_img = images_output_dir / img_path.name  # Save image in 'images' folder
        output_path_txt = labels_output_dir / img_path.with_suffix('.txt').name  # Save label in 'labels' folder
        predict_and_save(model, img_path, output_path_img, output_path_txt)

    print(f"Predicted images saved in {images_output_dir}")
    print(f"Bounding box labels saved in {labels_output_dir}")
    data = this_dir / 'yolo_params.yaml'
    print(f"Model parameters saved in {data}")
    metrics = model.val(data=data, split="test")

FileNotFoundError: [Errno 2] No such file or directory: 'runs/detect'

In [ ]:
# Train the model
model = YOLO('yolov8n.pt')  # Load a pretrained model
results = model.train(data='/content/Hackathon2_scripts_extracted/Hackathon2_scripts/yolo_params.yaml', epochs=10)  # Train the model

Ultralytics 8.3.202 🚀 Python-3.12.11 torch-2.8.0+cpu CPU (AMD EPYC 7B13)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Hackathon2_scripts_extracted/Hackathon2_scripts/yolo_params.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, p

In [ ]:
!pip install ultralytics
from ultralytics import YOLO
from pathlib import Path
import cv2
import os
import yaml



# Function to predict and save images
def predict_and_save(model, image_path, output_path, output_path_txt):
    # Perform prediction
    results = model.predict(image_path,conf=0.5)

    result = results[0]
    # Draw boxes on the image
    img = result.plot()  # Plots the predictions directly on the image

    # Save the result
    cv2.imwrite(str(output_path), img)
    # Save the bounding box data
    with open(output_path_txt, 'w') as f:
        for box in result.boxes:
            # Extract the class id and bounding box coordinates
            cls_id = int(box.cls)
            x_center, y_center, width, height = box.xywhn[0].tolist()

            # Write bbox information in the format [class_id, x_center, y_center, width, height]
            f.write(f"{cls_id} {x_center} {y_center} {width} {height}\n")


if __name__ == '__main__':

    this_dir = Path('.')
    os.chdir(this_dir)
    with open(this_dir / '/content/Hackathon2_scripts_extracted/Hackathon2_scripts/yolo_params.yaml', 'r') as file:
        data = yaml.safe_load(file)
        if 'test' in data and data['test'] is not None:
            images_dir = Path(data['test']) / 'images'
        else:
            print("No test field found in yolo_params.yaml, please add the test field with the path to the test images")
            exit()

    # check that the images directory exists
    if not images_dir.exists():
        print(f"Images directory {images_dir} does not exist")
        exit()

    if not images_dir.is_dir():
        print(f"Images directory {images_dir} is not a directory")
        exit()

    if not any(images_dir.iterdir()):
        print(f"Images directory {images_dir} is empty")
        exit()

    # Load the YOLO model
    detect_path = this_dir / "runs" / "detect"
    train_folders = [f for f in os.listdir(detect_path) if os.path.isdir(detect_path / f) and f.startswith("train")]
    if len(train_folders) == 0:
        raise ValueError("No training folders found")
    idx = 0
    if len(train_folders) > 1:
        choice = -1
        choices = list(range(len(train_folders)))
        while choice not in choices:
            print("Select the training folder:")
            for i, folder in enumerate(train_folders):
                print(f"{i}: {folder}")
            choice = input()
            if not choice.isdigit():
                choice = -1
            else:
                choice = int(choice)
        idx = choice

    model_path = detect_path / train_folders[idx] / "weights" / "best.pt"
    model = YOLO(model_path)

    # Directory with images
    output_dir = this_dir / "predictions" # Replace with the directory where you want to save predictions
    output_dir.mkdir(parents=True, exist_ok=True)

    # Create images and labels subdirectories
    images_output_dir = output_dir / 'images'
    labels_output_dir = output_dir / 'labels'
    images_output_dir.mkdir(parents=True, exist_ok=True)
    labels_output_dir.mkdir(parents=True, exist_ok=True)

    # Iterate through the images in the directory
    for img_path in images_dir.glob('*'):
        if img_path.suffix not in ['.png', '.jpg']:
            continue
        output_path_img = images_output_dir / img_path.name  # Save image in 'images' folder
        output_path_txt = labels_output_dir / img_path.with_suffix('.txt').name  # Save label in 'labels' folder
        predict_and_save(model, img_path, output_path_img, output_path_txt)

    print(f"Predicted images saved in {images_output_dir}")
    print(f"Bounding box labels saved in {labels_output_dir}")
    data = this_dir / 'yolo_params.yaml'
    print(f"Model parameters saved in {data}")
    metrics = model.val(data=data, split="test")